In [1]:
import joblib
import numpy as np 
import mne
import numpy as np
from scipy.stats import skew, kurtosis
from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
import pandas as pd
from scipy.fftpack import fft
import pywt  # For wavelet transform
import os

In [2]:
def preprocess_eeg(vhdr_file):
    """
    Preprocess EEG data with comprehensive artifact removal.
    
    This function combines the robust preprocessing pipeline from Method 1 with
    the extra processing steps from Method 2:
      - Additional artifact checks (ECG artifact removal)
      - Channel selection for ICA (only EEG and EOG channels)
      - Explicit reference projection

    Parameters
    ----------
    vhdr_file : str
        Path to BrainVision header file.
    save_dir : str or None
        Directory to save preprocessed data plots (optional).

    Returns
    -------
    raw : mne.io.Raw
        Preprocessed EEG data with artifacts removed.
    """
    # 1. Load the raw data
    raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)    
    # 3. Drop unnecessary channels if they exist
    if 'Mass' in raw.ch_names:
        raw.drop_channels('Mass')
    
    # 4. Set non-EEG channel types if available
    non_eeg_channels = {
        'VPVA': 'eog',
        'VNVB': 'eog',
        'HPHL': 'eog',
        'HNHR': 'eog',
        'Erbs': 'ecg',
        'OrbOcc': 'emg',
    }
    channel_types = {ch: typ for ch, typ in non_eeg_channels.items() if ch in raw.ch_names}
    if channel_types:
        raw.set_channel_types(channel_types)
    
    # 5. Set the montage (use on_missing='ignore' to avoid errors for auxiliary channels)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, on_missing='ignore')
    
    # 6. Apply a notch filter (50 Hz) to EEG channels and bandpass filter (0.1 - 40 Hz)
    raw.notch_filter(freqs=50, picks='eeg')
    raw.filter(l_freq=0.1, h_freq=40.0)
    
    # 7. Set average reference with projection and apply the projection
    raw.set_eeg_reference('average', projection=True)
    raw = raw.apply_proj()
    
    # 8. Instantiate the ICA model (with additional iterations for convergence)
    ica = ICA(n_components=20, random_state=42, max_iter=800)
    
    # 9. Pick only EEG and EOG channels for ICA fitting to reduce computational load
    eeg_picks = mne.pick_types(raw.info, eeg=True, eog=True, exclude='bads')
    ica.fit(raw, picks=eeg_picks)
    
    # --- Extra Processing Steps ---
    # ECG artifact removal: detect and exclude ECG-related ICA components.
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, ch_name='Erbs')
    ica.exclude = ecg_indices  # Start exclusion list with ECG-related components
    
    # EOG artifact removal: detect and add EOG-related ICA components to the exclusion list.
    eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=['VPVA', 'VNVB', 'HPHL', 'HNHR'])
    ica.exclude.extend(eog_indices)

    # 10. Apply ICA to remove the identified artifact components (modifies raw in place)
    raw = ica.apply(raw)
    # print(f"Preprocessing complete.")
    return raw


# Updated preprocessing

In [3]:
import mne
from mne.preprocessing import ICA
import warnings

def preprocess_eeg(vhdr_file):
    """
    Preprocess EEG data: load, filter, reference, run ICA for artifact removal.

    Parameters
    ----------
    vhdr_file : str
        Path to the BrainVision .vhdr file.

    Returns
    -------
    raw_clean : mne.io.Raw | None
        Preprocessed EEG data as an MNE Raw object. Returns None if an error occurs.
    """
    print(f"Processing: {vhdr_file}")
    try:
        # 1. Load raw EEG data
        raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose='warning')

        # 2. Drop unnecessary channels
        channels_to_drop = ['Mass', 'REF']
        existing_to_drop = [ch for ch in channels_to_drop if ch in raw.ch_names]
        if existing_to_drop:
            print(f"Dropping channels: {existing_to_drop}")
            raw.drop_channels(existing_to_drop)

        # 3. Ensure EEG channels exist
        if len(mne.pick_types(raw.info, eeg=True)) == 0:
            warnings.warn(f"No EEG channels in {vhdr_file}. Skipping.")
            return None

        # 4. Set non-EEG channel types
        non_eeg_channels = {
            'VPVA': 'eog', 'VNVB': 'eog',
            'HPHL': 'eog', 'HNHR': 'eog',
            'Erbs': 'ecg', 'OrbOcc': 'emg'
        }
        channel_types = {ch: typ for ch, typ in non_eeg_channels.items() if ch in raw.ch_names}
        if channel_types:
            print(f"Setting channel types: {channel_types}")
            raw.set_channel_types(channel_types)

        # 5. Apply montage
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage, on_missing='warn')

        # 6. Filtering
        print("Applying Notch filter (50 Hz)...")
        raw.notch_filter(freqs=50, picks='eeg', verbose='warning')
        print("Applying Bandpass filter (0.1 - 40 Hz)...")
        raw.filter(l_freq=0.1, h_freq=40.0, picks='eeg', verbose='warning')

        # 7. Average reference
        print("Setting average EEG reference...")
        raw.set_eeg_reference('average', projection=True)
        raw.apply_proj()

        # 8. ICA preparation
        n_components = 20
        picks_eeg = mne.pick_types(raw.info, eeg=True, exclude='bads')
        min_samples = n_components * 15
        if raw.n_times < min_samples:
            warnings.warn(f"Not enough data for ICA ({raw.n_times} < {min_samples}). Skipping ICA.")
            return raw

        ica = ICA(n_components=n_components, random_state=42, max_iter=800)
        print(f"Fitting ICA with {n_components} components...")
        try:
            ica.fit(raw, picks=picks_eeg, reject_by_annotation=False)
        except Exception as e:
            warnings.warn(f"ICA fitting failed: {e}")
            return raw

        # 9. Detect ECG artifacts
        ica.exclude = []
        if 'Erbs' in raw.ch_names:
            try:
                ecg_inds, scores_ecg = ica.find_bads_ecg(raw, ch_name='Erbs', method='ctps', threshold=0.23)
                if ecg_inds:
                    print(f"ECG components: {ecg_inds}")
                    ica.exclude.extend(ecg_inds)
            except Exception as e:
                warnings.warn(f"find_bads_ecg failed: {e}")
        else:
            print("No ECG channel found. Skipping ECG artifact detection.")

        # 10. Detect EOG artifacts
        eog_channels = [ch for ch in ['VPVA', 'VNVB', 'HPHL', 'HNHR'] if ch in raw.ch_names]
        if eog_channels:
            try:
                eog_inds, scores_eog = ica.find_bads_eog(raw, ch_name=eog_channels, threshold=3.0)
                new_inds = [i for i in eog_inds if i not in ica.exclude]
                if new_inds:
                    print(f"EOG components: {new_inds}")
                    ica.exclude.extend(new_inds)
            except Exception as e:
                warnings.warn(f"find_bads_eog failed: {e}")
        else:
            print("No EOG channels found. Skipping EOG artifact detection.")

        print(f"Excluding components: {ica.exclude}")
        raw_clean = ica.apply(raw.copy(), exclude=ica.exclude)

    except Exception as e:
        warnings.warn(f"Processing error: {e}")
        return None

    print(f"Finished processing: {vhdr_file}")
    return raw_clean


In [ ]:
raw = mne.io.read_raw_brainvision("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr", preload=True)

In [ ]:
raw = preprocess_eeg("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr")

In [4]:
import os
import mne
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

def slice_and_save_eeg(vhdr_file_path, output_dir, slice_duration=60):
    try:
        base_name = os.path.splitext(os.path.basename(vhdr_file_path))[0]
        output_file_1 = os.path.join(output_dir, f"{base_name}_1_eeg.fif")

        # Skip if already processed
        if os.path.exists(output_file_1):
            print(f"Skipping {base_name} (already processed).")
            return

        raw = preprocess_eeg(vhdr_file_path)
        sfreq = raw.info['sfreq']
        total_duration = raw.n_times / sfreq

        if total_duration <= 0:
            print(f"Skipping {base_name}: empty data.")
            return

        if total_duration >= slice_duration:
            # First slice: 0 to 60 seconds
            slice1 = raw.copy().crop(tmin=0, tmax=slice_duration, include_tmax=False)
            slice1.save(output_file_1, overwrite=True)
            print(f"Saved {base_name}_1 (0–60s)")

            if total_duration > slice_duration + 1.0:
                output_file_2 = os.path.join(output_dir, f"{base_name}_2_eeg.fif")
                slice2 = raw.copy().crop(tmin=slice_duration + 1.0, tmax=None)
                slice2.save(output_file_2, overwrite=True)
                print(f"Saved {base_name}_2 (61s–end)")
            else:
                print(f"Not enough data for {base_name}_2: total duration = {total_duration:.2f}s")
        else:
            raw.save(output_file_1, overwrite=True)
            print(f"Saved {base_name}_1 (<60s)")

    except Exception as e:
        print(f"Error processing {vhdr_file_path}: {e}")
        traceback.print_exc()


In [ ]:
slice_and_save_eeg("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEO_eeg.vhdr","../")

In [ ]:
slice_and_save_eeg("/mnt/data/saikrishna/Team_4/dataset_s/mdd2/sub-88005893/ses-1/eeg/sub-88005893_ses-1_task-restEO_eeg.vhdr","../")

In [ ]:
raw = mne.io.read_raw_fif("../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif", preload=True)

In [ ]:
print("All slices saved successfully.")

In [5]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            
            for file in os.listdir(session_path):
                if file.endswith("_eeg.vhdr"):
                    inp_path = os.path.join(session_path, file)
                    output_filename = f"{file.replace('.vhdr', '.fif')}"
                    # output_filename = output_filename.replace("task-restcombined", "task-rest_combined")
                    output_path = os.path.join(destination_folder, output_filename)
                    slice_and_save_eeg(inp_path,destination_folder)



# Improved Processes_folder

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_single_file_wrapper(vhdr_file_path, output_dir):
    # Calls your existing slice_and_save_eeg() with error handling
    slice_and_save_eeg(vhdr_file_path, output_dir)

def process_folder(source_folder, destination_folder, max_workers=6, slice_duration=60):
    """
    Processes all *_eeg.vhdr EEG files in nested subject/session folders using slice_and_save_eeg.
    
    Args:
        source_folder (str): Root folder with subject/session/eeg structure.
        destination_folder (str): Folder where sliced FIF files will be saved.
        max_workers (int): Number of threads to use for parallel processing.
        slice_duration (int): Duration in seconds for the first slice.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    vhdr_files_to_process = []

    # Traverse the folder structure to find all *_eeg.vhdr files
    for subject in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, subject)
        if not os.path.isdir(subject_path):
            continue

        for session in os.listdir(subject_path):
            session_path = os.path.join(subject_path, session, "eeg")
            if not os.path.isdir(session_path):
                continue

            for file in os.listdir(session_path):
                if file.endswith("_eeg.vhdr"):
                    vhdr_path = os.path.join(session_path, file)
                    base_name = os.path.splitext(file)[0]
                    output_file = os.path.join(destination_folder, f"{base_name}_1_eeg.fif")
                    
                    if not os.path.exists(output_file):
                        vhdr_files_to_process.append(vhdr_path)
                    else:
                        print(f"Skipping {base_name} (already processed)")

    print(f"Found {len(vhdr_files_to_process)} files to process.")

    # Parallel processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(slice_and_save_eeg, vhdr_path, destination_folder)
            for vhdr_path in vhdr_files_to_process
        ]
        for _ in as_completed(futures):
            pass  # Progress handled by slice_and_save_eeg's print statements


In [ ]:
raw = mne.io.read_raw_fif("/mnt/data/saikrishna/Team_4/split_fif_new/mdd2/sub-88005893_ses-1_task-restEC_eeg_1_eeg.fif", preload=True)


In [ ]:
raw.load_data()


In [7]:
process_folder("../../dataset_s/mdd3","../../split_fif_new/mdd3")

Processing: ../../dataset_s/mdd3/sub-88042969/ses-1/eeg/sub-88042969_ses-1_task-restEC_eeg.vhdr
Dropping channels: ['Mass']
Setting channel types: {'VPVA': 'eog', 'VNVB': 'eog', 'HPHL': 'eog', 'HNHR': 'eog', 'Erbs': 'ecg', 'OrbOcc': 'emg'}
Applying Notch filter (50 Hz)...
Applying Bandpass filter (0.1 - 40 Hz)...
Setting average EEG reference...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Fitting ICA with 20 components...
Fitting ICA to data using 26 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 20 components
    Applying projection 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm a

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
EOG components: [np.int64(0)]
Excluding components: [np.int64(0)]
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 1 ICA component
    Projecting back 

...

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
Excluding components: [np.int64(6)]
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 1 ICA component
    Projecting back using 26 PCA components
Finished processing: ../../dataset_s/mdd3/sub-88053317/ses-1/eeg/sub-88053317_ses-1_task-restEO_eeg.vhdr
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEO_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEO_eeg_1_eeg.fif
[done]
Saved sub-88053317_ses-1_task-restEO_eeg_1 (0–60s)
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEO_eeg

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann win

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
Excluding components: []
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 26 PCA components
Finished processing: ../../dataset_s/mdd3/sub-88053317/ses-1/eeg/sub-88053317_ses-1_task-restEC_eeg.vhdr
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEC_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEC_eeg_1_eeg.fif
[done]
Saved sub-88053317_ses-1_task-restEC_eeg_1 (0–60s)
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd3/sub-88053317_ses-1_task-restEC_eeg_2_eeg.fif

In [6]:
process_folder("../../dataset_s/mdd2","../../split_fif_new/mdd2")

Skipping sub-88032841_ses-1_task-restEC_eeg (already processed).
Skipping sub-88032841_ses-1_task-restEO_eeg (already processed).
Skipping sub-88005893_ses-1_task-restEC_eeg (already processed).
Skipping sub-88005893_ses-1_task-restEO_eeg (already processed).
Skipping sub-88018941_ses-1_task-restEC_eeg (already processed).
Skipping sub-88018941_ses-1_task-restEO_eeg (already processed).
Skipping sub-88029645_ses-1_task-restEO_eeg (already processed).
Skipping sub-88029645_ses-1_task-restEC_eeg (already processed).
Skipping sub-88012817_ses-1_task-restEO_eeg (already processed).
Skipping sub-88012817_ses-1_task-restEC_eeg (already processed).
Skipping sub-88030369_ses-1_task-restEC_eeg (already processed).
Skipping sub-88030369_ses-1_task-restEO_eeg (already processed).
Skipping sub-88030101_ses-1_task-restEC_eeg (already processed).
Skipping sub-88030101_ses-1_task-restEO_eeg (already processed).
Skipping sub-88025057_ses-1_task-restEC_eeg (already processed).
Skipping sub-88025057_ses

/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Number of ECG events detected : 118 (average pulse 58.98230530840748 / min.)
Not setting metadata
118 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 118 events and 501 original time points ...
1 bad epochs dropped
    Applying projection operator with 1 vector (pre-whitener application)
Using EOG channels: VPVA, VNVB, HPHL, HNHR
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff fr

...

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
Excluding components: []
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 26 PCA components
Finished processing: ../../dataset_s/mdd2/sub-88033561/ses-1/eeg/sub-88033561_ses

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm a

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


joblib supports memapping pool but "MNE_CACHE_DIR" is not set in MNE-Python config. To enable it, use, e.g., mne.set_cache_dir('/tmp/shm'). This will store temporary files under /dev/shm and can result in large memory savings.
Excluding components: []
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 26 PCA components
Finished processing: ../../dataset_s/mdd2/sub-88033561/ses-1/eeg/sub-88033561_ses-1_task-restEC_eeg.vhdr
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd2/sub-88033561_ses-1_task-restEC_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd2/sub-88033561_ses-1_task-restEC_eeg_1_eeg.fif
[done]
Saved sub-88033561_ses-1_task-restEC_eeg_1 (0–60s)
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd2/sub-88033561_ses-1_task-restEC_eeg_2_eeg.fif

In [ ]:
import os
import mne

def slice_and_save_missing_second_slice(vhdr_file_path, output_dir, slice_duration=60):
    base_name = os.path.splitext(os.path.basename(vhdr_file_path))[0]
    second_slice_path = os.path.join(output_dir, f"{base_name}_2_eeg.fif")
    
    if os.path.exists(second_slice_path):
        print(f"[✓] Already exists: {second_slice_path}")
        return

    print(f"[→] Recomputing second slice for: {vhdr_file_path}")
    raw = preprocess_eeg(vhdr_file_path)
    sfreq = raw.info['sfreq']
    samples_per_slice = int(slice_duration * sfreq)

    start_sample = samples_per_slice
    end_sample = 2 * samples_per_slice

    if end_sample > len(raw):
        print(f"[!] Not enough data for second slice in: {vhdr_file_path}")
        return

    tmin = start_sample / sfreq
    tmax = min(end_sample / sfreq, raw.times[-1] - 1e-6)  # <-- Safe cap

    sliced_raw = raw.copy().crop(tmin=tmin, tmax=tmax, include_tmax=False)
    sliced_raw.save(second_slice_path, overwrite=True)
    print(f"[+] Saved second slice: {second_slice_path}")


def process_for_missing_second_slices(source_folder, destination_folder):
    """
    Scans all EO EEG files and recomputes missing _2_eeg.fif slices if needed.
    """
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue

        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue

            for file in os.listdir(session_path):
                if file.endswith("_eeg.vhdr") and "restEO" in file:
                    inp_path = os.path.join(session_path, file)
                    slice_and_save_missing_second_slice(inp_path, destination_folder)


In [ ]:
import os
from os.path import join

def check_and_recompute_missing_slices(source_folder, destination_folder, slice_duration=60):
    """
    Check if all four required slices are present for each subject and recompute any missing slices.
    
    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
        slice_duration (int): Duration of each slice in seconds (default is 60).
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Walk through all subject directories
    for sub_id in os.listdir(source_folder):
        subject_path = join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue

        # Check for both EO and EC sessions for the subject
        eo_files = []
        ec_files = []

        # Collect all the relevant files for this subject
        for file in os.listdir(subject_path):
            if file.endswith("_eeg.fif"):
                if "restEO" in file:
                    eo_files.append(file)
                elif "restEC" in file:
                    ec_files.append(file)

        # Check if all four files (EO and EC) exist
        expected_eo_files = ['restEO_eeg_1_eeg.fif', 'restEO_eeg_2_eeg.fif']
        expected_ec_files = ['restEC_eeg_1_eeg.fif', 'restEC_eeg_2_eeg.fif']
        
        missing_files = False
        for file in expected_eo_files + expected_ec_files:
            if file not in eo_files and file not in ec_files:
                missing_files = True
                break
        
        # If all files are present, skip the subject
        if not missing_files:
            print(f"[✓] All files present for subject {sub_id}. Skipping.")
            continue

        # Recompute missing files
        print(f"[→] Recomputing missing slices for subject {sub_id}...")

        # Recompute each missing file
        for eo_file in expected_eo_files:
            if eo_file not in eo_files:
                # Recompute the missing EO slice
                print(f"Recomputing missing file {eo_file} for subject {sub_id}...")
                inp_path = join(subject_path, eo_file.replace("restEO", "restEO"))
                recompute_missing_first_slice(inp_path, destination_folder, slice_duration)

        for ec_file in expected_ec_files:
            if ec_file not in ec_files:
                # Recompute the missing EC slice
                print(f"Recomputing missing file {ec_file} for subject {sub_id}...")
                inp_path = join(subject_path, ec_file.replace("restEC", "restEC"))
                recompute_missing_first_slice(inp_path, destination_folder, slice_duration)

def recompute_missing_first_slice(vhdr_file_path, output_dir, slice_duration=60):
    """Recompute the missing first slice (either EO or EC)."""
    # Make sure we're dealing with the .vhdr file (not .fif)
    base_name = os.path.splitext(os.path.basename(vhdr_file_path))[0]
    
    # Perform preprocessing with the correct .vhdr file
    raw = preprocess_eeg(vhdr_file_path)  # Ensure this is the .vhdr file

    # Get sampling frequency
    sfreq = raw.info['sfreq']
    
    # Calculate the number of samples per slice
    samples_per_slice = int(slice_duration * sfreq)
    
    # Calculate the start and end sample for the first slice
    start_sample = 0
    end_sample = samples_per_slice
    
    # Create a new raw object for the first slice
    sliced_raw = raw.copy().crop(
        tmin=start_sample / sfreq, 
        tmax=end_sample / sfreq, 
        include_tmax=False
    )
    
    # Construct the filename for the first slice
    slice_filename = f"{base_name}_1_eeg.fif"
    
    # Save the first slice in the output directory
    slice_output_path = os.path.join(output_dir, slice_filename)
    sliced_raw.save(slice_output_path, overwrite=True)
    print(f"[+] Saved first slice: {slice_output_path}")


# Example usage
check_and_recompute_missing_slices("../../dataset_s/mdd2", "../../split_fif_new/mdd2")


In [ ]:
process_for_missing_second_slices("../../dataset_s/mdd2","../../split_fif_new/mdd2")

In [ ]:
process_folder("../../dataset_s/mdd","../../split_fif_new/mdd")

In [ ]:
process_folder("../../dataset_s/mdd2","../../split_fif_new/mdd2")

In [ ]:
process_folder("../../dataset_s/healthy","../../split_fif_new/healthy")

# END

In [ ]:
def extract_channel_features(raw, fmin=0.5, fmax=50):
    # Select only EEG channels
    raw.pick_types(eeg=True)  # This removes non-EEG channels
    data = raw.get_data()
    channel_names = raw.ch_names
    features = {ch: {} for ch in channel_names}

    # Time-domain features
    for i, ch in enumerate(channel_names):
        features[ch]['mean'] = np.mean(data[i])
        features[ch]['variance'] = np.var(data[i])
        features[ch]['skewness'] = skew(data[i])
        features[ch]['kurtosis'] = kurtosis(data[i])
        features[ch]['peak_to_peak'] = np.ptp(data[i])

        # Fourier Transform (FFT)
        fft_values = np.abs(fft(data[i]))
        features[ch]['fft_mean'] = np.mean(fft_values)
        features[ch]['fft_std'] = np.std(fft_values)
        features[ch]['fft_max'] = np.max(fft_values)
        
        # Wavelet Transform (Morlet)
        wavelet = 'cmor1.5-1.0'
        coeffs, _ = pywt.cwt(data[i], scales=np.arange(1, 129), wavelet=wavelet)
        coeffs = np.abs(coeffs)  # Convert complex values to magnitude
        
        features[ch]['wavelet_energy'] = np.sum(np.square(coeffs))

        # Wavelet Transform (DWT) using Daubechies wavelet (db4) #morle
        # coeffs = pywt.wavedec(data[i], 'db4', level=4)
        # features[ch]['wavelet_energy'] = sum(np.sum(np.square(c)) for c in coeffs)
        
        # features[ch]['wavelet_entropy'] = 0  # Initialize wavelet_entropy
        
        # for c in coeffs:
        #     c = c[np.isfinite(c)]
        #     c_norm = c / (np.sum(np.abs(c)) + 1e-10)
        #     features[ch]['wavelet_entropy'] += -np.sum(c_norm * np.log2(c_norm + 1e-10))

    # Frequency-domain features using PSD
    psd = raw.compute_psd(method='welch', fmin=fmin, fmax=fmax, n_fft=2048)
    psd_data = psd.get_data()
    freqs = psd.freqs
    psd_df = pd.DataFrame(psd_data, columns=freqs, index=channel_names)

    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'slow_alpha': (6, 9), 'alpha': (8, 12),
             'beta': (12, 30), 'gamma': (30, 50)}

    for band, (low, high) in bands.items():
        band_power = psd_df.loc[:, (freqs >= low) & (freqs <= high)].mean(axis=1)
        for ch in channel_names:
            features[ch][f'{band}_power'] = band_power[ch]

    # Frontal Alpha Asymmetry (F3-F4)
    if 'F3' in channel_names and 'F4' in channel_names:
        features['F3_F4_alpha_asymmetry'] = features['F4']['alpha_power'] - features['F3']['alpha_power']

    # Convert features to DataFrame
    features_df = pd.DataFrame(features).T

    return features_df

In [ ]:
def process_and_combine(eo_file_path, ec_file_path, output_file):
    all_features = []

    # Process EO file
    raw_eo = mne.io.read_raw_fif(eo_file_path)
    features_eo = extract_channel_features(raw_eo)
    features_eo['condition'] = 'EO'
    all_features.append(features_eo)

    # Process EC file
    raw_ec = mne.io.read_raw_fif(ec_file_path)
    features_ec = extract_channel_features(raw_ec)
    features_ec['condition'] = 'EC'
    all_features.append(features_ec)

    # Combine EO and EC features
    combined_features = pd.concat(all_features, keys=['EO', 'EC'], names=['condition', 'channel'])
    
    # Save combined features to a single CSV file
    combined_features.to_csv(output_file)
    print(f"Features successfully saved to {output_file}")
    # return combined_features

In [ ]:
process_and_combine("split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.csv")

In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    output_filename=""
    eo_path=""
    ec_path=""
    print(os.listdir(source_folder))
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            print(ses_id)
            for file in os.listdir(session_path):
                if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
                    eo_path = os.path.join(session_path, file)
                elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
                    ec_file=file
                    ec_path = os.path.join(session_path, file)
                
                print(eo_path,ec_path)
                output_filename=ec_file.replace("EC_eeg","_eeg_combined.csv")
                process_and_combine(eo_path,ec_path,output_filename)

                
                
                # output_path = os.path.join(destination_folder, output_filename)
                




In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO and EC files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    eo_path, ec_path = None, None
    ec_file = None  # Initialize ec_file to avoid the UnboundLocalError

    # Iterate through files in the source folder
    for file in os.listdir(source_folder):
        file_path = os.path.join(source_folder, file)

        if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
            ec_path = file_path
            ec_file = file  # Store EC file name for output filename generation
        elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
            eo_path = file_path

        # Process only when both EO and EC files are found
        if eo_path and ec_path and ec_file:
            output_filename = ec_file.replace("EC_eeg", "eeg_combined.csv")
            output_filepath = os.path.join(destination_folder, output_filename)

            print(f"Processing: \n  EO: {eo_path} \n  EC: {ec_path} \n  Output: {output_filepath}")
            process_and_combine(eo_path, ec_path, output_filepath)

            # Reset paths after processing
            eo_path, ec_path, ec_file = None, None, None


In [ ]:
process_folder("dataset_s/mdd","split_fif/mdd")